In [2]:
import tkinter as tk

# handling button click event
def button_click():
    #print("Button clicked!")

    #show an ainfo message box
    msgbox.showinfo("Info, Welcome to our COS102 project 'Movie booking'!\n")

    #ask for user confirmation
    result = msgbox.askyesno("Confirmation", "Do you want to continue?")

#creating main window
root = tk.Tk()
root.title("Welcome to our cinema")
root.geometry("300x100")

# add a labe widget
label = tk.Label(root, text="Hello friend \n")
label.pack()

# add a button widget
button = tk.Button(root, text="Click Me", command=button_click)
button.pack()

#styling the button
button.config(fg="orange", bg="red")

#start event loop
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ISMS\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\ISMS\AppData\Local\Temp\ipykernel_39356\991011621.py", line 8, in button_click
    msgbox.showinfo("Info, Welcome to our COS102 project 'Movie booking'!\n")
    ^^^^^^
NameError: name 'msgbox' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ISMS\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\ISMS\AppData\Local\Temp\ipykernel_39356\991011621.py", line 8, in button_click
    msgbox.showinfo("Info, Welcome to our COS102 project 'Movie booking'!\n")
    ^^^^^^
NameError: name 'msgbox' is not defined


In [ ]:
import tkinter as tk
from tkinter import messagebox

def welcomeMessage(Uusername):
    #create a Tkinter window
    window = tk.Toplevel(root)
    window.title("Admin Box")
    window.geometry("500x200")

    label_1 = tk.Label(window, text=f"Welcome {username}\n")
    label_1.pack()
    label_2 = tk.Label(window, text="This is a movie layout youre seeing (Be creative bro.....)")
    label_2.pack()

    #run the Tkinter event loop
    root.mainloop()

    def submit():
        username = username_entry.get()
        password = password_entry.get()

        if username == "Mary" and password == "cos102":
            WelcomeMessage(username)
        else:
            msgbox.showerror("Login", "Invalid username or password")

#create main window
root = tk.Tk()
root.title("Login Form")
root.geometry("500x200")


# Create username label and entry
usermane_label = tk.Label(root, text="Username:")
usermane_label.pack()
username_entry = tk.Entry(root)
username_entry.pack()

# Create Password label and entry
password_label = tk.Label(root, text="Password:")
password_label.pack()
password_entry = tk.Entry(root, show="*")
password_entry.pack()

#Create submit button
submit_button = tk.Button(root, text="Submit", command= "submit")
submit_button.pack()

#run the main event loop
root.mainloop()


In [13]:
#!/usr/bin/env python3
"""
Panaview Cinemas Movie Ticket Booking System
A professional-grade desktop application for managing movie ticket bookings with Tkinter and SQLite.
Meets requirements for movie selection, showtime selection, seat booking, and persistent storage.
"""

import tkinter as tk
from tkinter import messagebox, ttk
import sqlite3
from datetime import datetime
import logging
import uuid
from typing import List, Tuple, Optional

# Configure logging for debugging and monitoring
logging.basicConfig(
    level=logging.INFO,
    filename="cinema_app.log",
    format="%(asctime)s - %(levelname)s - %(message)s"
)

class Seat:
    """Represents a seat in the cinema with row, number, and status."""
    def __init__(self, row: int, number: int, status: str = "available"):
        self.row = row
        self.number = number
        self.status = status  # available, selected, booked

    def set_status(self, status: str) -> None:
        """Update the seat's status."""
        if status in ["available", "selected", "booked"]:
            self.status = status
        else:
            raise ValueError(f"Invalid seat status: {status}")

    def is_available(self) -> bool:
        """Check if the seat is available."""
        return self.status == "available"

    def __str__(self) -> str:
        return f"Seat {chr(65 + self.row)}{self.number + 1} ({self.status})"

class Booking:
    """Represents a booking with movie, showtime, and seat details."""
    def __init__(self, movie_title: str, showtime: str, seat: Seat, user_name: Optional[str] = None):
        self.movie_title = movie_title
        self.showtime = showtime
        self.seat = seat
        self.user_name = user_name or "Guest"
        self.timestamp = datetime.now().isoformat()

class DatabaseManager:
    """Manages SQLite database operations for movies, showtimes, and bookings."""
    def __init__(self, db_name: str):
        try:
            self.conn = sqlite3.connect(db_name, check_same_thread=False)
            self.cursor = self.conn.cursor()
            self.create_tables()
            logging.info("Database initialized successfully")
        except sqlite3.Error as e:
            logging.error(f"Database initialization failed: {e}")
            raise

    def create_tables(self) -> None:
        """Create database tables with appropriate schema."""
        try:
            self.cursor.execute('''
                CREATE TABLE IF NOT EXISTS movies (
                    title TEXT PRIMARY KEY,
                    duration INTEGER NOT NULL
                )
            ''')
            self.cursor.execute('''
                CREATE TABLE IF NOT EXISTS showtimes (
                    id TEXT PRIMARY KEY,
                    movie_title TEXT NOT NULL,
                    showtime TEXT NOT NULL,
                    FOREIGN KEY (movie_title) REFERENCES movies(title)
                )
            ''')
            self.cursor.execute('''
                CREATE TABLE IF NOT EXISTS bookings (
                    id TEXT PRIMARY KEY,
                    movie_title TEXT NOT NULL,
                    showtime_id TEXT NOT NULL,
                    row INTEGER NOT NULL,
                    seat_num INTEGER NOT NULL,
                    user_name TEXT NOT NULL,
                    timestamp TEXT NOT NULL,
                    FOREIGN KEY (movie_title) REFERENCES movies(title),
                    FOREIGN KEY (showtime_id) REFERENCES showtimes(id)
                )
            ''')
            self.conn.commit()
        except sqlite3.Error as e:
            logging.error(f"Table creation failed: {e}")
            raise

    def add_movie(self, title: str, duration: int) -> None:
        """Add a movie to the database."""
        try:
            self.cursor.execute("INSERT OR IGNORE INTO movies VALUES (?, ?)", (title, duration))
            self.conn.commit()
            logging.info(f"Added movie: {title}")
        except sqlite3.Error as e:
            logging.error(f"Failed to add movie {title}: {e}")

    def add_showtime(self, movie_title: str, showtime: str) -> None:
        """Add a showtime for a movie."""
        try:
            showtime_id = str(uuid.uuid4())
            self.cursor.execute("INSERT OR IGNORE INTO showtimes VALUES (?, ?, ?)",
                              (showtime_id, movie_title, showtime))
            self.conn.commit()
            logging.info(f"Added showtime {showtime} for {movie_title}")
        except sqlite3.Error as e:
            logging.error(f"Failed to add showtime {showtime} for {movie_title}: {e}")

    def get_movies(self) -> List[str]:
        """Retrieve all movie titles."""
        try:
            self.cursor.execute("SELECT title FROM movies")
            return [row[0] for row in self.cursor.fetchall()]
        except sqlite3.Error as e:
            logging.error(f"Failed to retrieve movies: {e}")
            return []

    def get_showtimes(self, movie_title: str) -> List[Tuple[str, str]]:
        """Retrieve showtimes for a movie with their IDs."""
        try:
            self.cursor.execute("SELECT id, showtime FROM showtimes WHERE movie_title = ?",
                              (movie_title,))
            return self.cursor.fetchall()
        except sqlite3.Error as e:
            logging.error(f"Failed to retrieve showtimes for {movie_title}: {e}")
            return []

    def save_booking(self, booking: Booking) -> None:
        """Save a booking to the database."""
        try:
            showtime_id = self.get_showtime_id(booking.movie_title, booking.showtime)
            booking_id = str(uuid.uuid4())
            self.cursor.execute(
                "INSERT INTO bookings VALUES (?, ?, ?, ?, ?, ?, ?)",
                (booking_id, booking.movie_title, showtime_id, booking.seat.row,
                 booking.seat.number, booking.user_name, booking.timestamp)
            )
            self.conn.commit()
            logging.info(f"Saved booking for {booking.movie_title} at {booking.showtime}, "
                        f"seat {chr(65 + booking.seat.row)}{booking.seat.number + 1}")
        except sqlite3.Error as e:
            logging.error(f"Failed to save booking: {e}")
            raise

    def get_booked_seats(self, movie_title: str, showtime: str) -> List[Tuple[int, int]]:
        """Retrieve booked seats for a movie and showtime."""
        try:
            showtime_id = self.get_showtime_id(movie_title, showtime)
            self.cursor.execute(
                "SELECT row, seat_num FROM bookings WHERE movie_title = ? AND showtime_id = ?",
                (movie_title, showtime_id)
            )
            return [(row[0], row[1]) for row in self.cursor.fetchall()]
        except sqlite3.Error as e:
            logging.error(f"Failed to retrieve booked seats for {movie_title}, {showtime}: {e}")
            return []

    def get_showtime_id(self, movie_title: str, showtime: str) -> str:
        """Retrieve the showtime ID for a given movie and showtime."""
        try:
            self.cursor.execute(
                "SELECT id FROM showtimes WHERE movie_title = ? AND showtime = ?",
                (movie_title, showtime)
            )
            result = self.cursor.fetchone()
            if result:
                return result[0]
            raise ValueError(f"No showtime found for {movie_title} at {showtime}")
        except sqlite3.Error as e:
            logging.error(f"Failed to get showtime ID: {e}")
            raise

    def close_connection(self) -> None:
        """Close the database connection."""
        try:
            self.conn.close()
            logging.info("Database connection closed")
        except sqlite3.Error as e:
            logging.error(f"Failed to close database connection: {e}")

class CinemaApp:
    """Main application class for the cinema booking system."""
    ROWS = 5
    COLS = 10

    def __init__(self):
        try:
            self.root = tk.Tk()
            self.root.title("Panaview Cinemas Booking System")
            self.root.geometry("900x700")
            self.root.resizable(False, False)
            self.root.configure(bg="#f0f0f0")
            self.root.protocol("WM_DELETE_WINDOW", self.close_app)
            self.db = DatabaseManager("cinema.db")
            self.seats: dict = {}  # (row, col): Button
            self.selected_seats: List[Tuple[int, int]] = []
            self.seat_layout: List[List[Seat]] = []
            self.user_name = tk.StringVar(value="Guest")
            self.initialize_data()
            self.create_gui()
            logging.info("Application initialized successfully")
        except Exception as e:
            logging.error(f"Application initialization failed: {e}")
            messagebox.showerror("Error", f"Failed to start application: {e}")
            self.root.destroy()

    def initialize_data(self) -> None:
        """Initialize sample data for testing."""
        try:
            movies = [
                ("Avengers: Endgame", 180),
                ("The Matrix", 136),
                ("Inception", 148)
            ]
            showtimes = {
                "Avengers: Endgame": ["10:00 AM", "2:00 PM", "6:00 PM"],
                "The Matrix": ["12:00 PM", "4:00 PM"],
                "Inception": ["11:00 AM", "3:00 PM"]
            }
            for title, duration in movies:
                self.db.add_movie(title, duration)
            for movie, times in showtimes.items():
                for time in times:
                    self.db.add_showtime(movie, time)
            logging.info("Sample data initialized")
        except Exception as e:
            logging.error(f"Data initialization failed: {e}")

    def create_gui(self) -> None:
        """Create the main GUI layout."""
        try:
            style = ttk.Style()
            style.configure("TLabel", font=("Arial", 12), background="#f0f0f0")
            style.configure("TButton", font=("Arial", 12))
            style.configure("TEntry", font=("Arial", 12))

            # User name input
            ttk.Label(self.root, text="Your Name:").pack(pady=5)
            ttk.Entry(self.root, textvariable=self.user_name, width=30).pack(pady=5)

            # Movie selection
            ttk.Label(self.root, text="Select Movie:").pack(pady=5)
            self.movie_var = tk.StringVar()
            movies = self.db.get_movies()
            if not movies:
                messagebox.showerror("Error", "No movies available. Please contact support.")
                self.root.destroy()
                return
            self.movie_var.set(movies[0])
            ttk.OptionMenu(self.root, self.movie_var, movies[0], *movies,
                          command=self.on_movie_select).pack(pady=5)

            # Showtime selection
            ttk.Label(self.root, text="Select Showtime:").pack(pady=5)
            self.showtime_var = tk.StringVar()
            self.showtime_menu = ttk.OptionMenu(self.root, self.showtime_var, "",
                                               command=self.on_showtime_select)
            self.showtime_menu.pack(pady=5)

            # Seat layout frame
            self.seat_frame = ttk.Frame(self.root)
            self.seat_frame.pack(pady=10)

            # Screen label
            ttk.Label(self.root, text="SCREEN", font=("Arial", 14, "bold"),
                     background="#d3d3d3", padding=5).pack(fill="x", padx=20, pady=5)

            # Seat status legend
            legend_frame = ttk.Frame(self.root)
            legend_frame.pack(pady=5)
            ttk.Label(legend_frame, text="Available", background="green", foreground="white",
                     padding=3).pack(side="left", padx=5)
            ttk.Label(legend_frame, text="Selected", background="yellow", foreground="black",
                     padding=3).pack(side="left", padx=5)
            ttk.Label(legend_frame, text="Booked", background="red", foreground="white",
                     padding=3).pack(side="left", padx=5)

            # Confirm button
            ttk.Button(self.root, text="Confirm Booking", command=self.confirm_booking).pack(pady=10)

            # Initialize showtimes for default movie
            self.on_movie_select(None)
            logging.info("GUI created successfully")
        except Exception as e:
            logging.error(f"GUI creation failed: {e}")
            messagebox.showerror("Error", f"Failed to create interface: {e}")

    def on_movie_select(self, event: Optional[tk.Event]) -> None:
        """Handle movie selection and update showtimes."""
        try:
            self.showtime_var.set("")
            self.clear_seat_layout()
            showtimes = self.db.get_showtimes(self.movie_var.get())
            self.showtime_menu["menu"].delete(0, tk.END)
            if not showtimes:
                messagebox.showinfo("Info", "No showtimes available for this movie.")
                return
            for showtime_id, showtime in showtimes:
                self.showtime_menu["menu"].add_command(
                    label=showtime, command=lambda t=showtime: self.showtime_var.set(t)
                )
            self.showtime_var.set(showtimes[0][1])
            self.on_showtime_select(None)
        except Exception as e:
            logging.error(f"Movie selection failed: {e}")
            messagebox.showerror("Error", f"Failed to load showtimes: {e}")

    def on_showtime_select(self, event: Optional[tk.Event]) -> None:
        """Handle showtime selection and display seat layout."""
        try:
            if self.showtime_var.get():
                self.display_seat_layout()
        except Exception as e:
            logging.error(f"Showtime selection failed: {e}")
            messagebox.showerror("Error", f"Failed to load seat layout: {e}")

    def display_seat_layout(self) -> None:
        """Display the seat layout for the selected showtime."""
        try:
            self.clear_seat_layout()
            self.seat_layout = [[Seat(row, col) for col in range(self.COLS)]
                               for row in range(self.ROWS)]
            booked_seats = self.db.get_booked_seats(self.movie_var.get(), self.showtime_var.get())

            for row, col in booked_seats:
                if row < self.ROWS and col < self.COLS:
                    self.seat_layout[row][col].set_status("booked")

            for row in range(self.ROWS):
                for col in range(self.COLS):
                    seat = self.seat_layout[row][col]
                    bg_color = "green" if seat.is_available() else "red"
                    btn = ttk.Button(
                        self.seat_frame,
                        text=f"{chr(65 + row)}{col + 1}",
                        width=4,
                        style="TButton",
                        command=lambda r=row, c=col: self.on_seat_click(r, c)
                    )
                    btn.configure(style=f"{bg_color}.TButton")
                    btn.grid(row=row, column=col, padx=2, pady=2)
                    self.seats[(row, col)] = btn

            # Configure button styles for colors
            style = ttk.Style()
            style.configure("green.TButton", background="green", foreground="white")
            style.configure("yellow.TButton", background="yellow", foreground="black")
            style.configure("red.TButton", background="red", foreground="white")
            logging.info("Seat layout displayed")
        except Exception as e:
            logging.error(f"Seat layout display failed: {e}")
            messagebox.showerror("Error", f"Failed to display seats: {e}")

    def on_seat_click(self, row: int, col: int) -> None:
        """Handle seat selection toggle."""
        try:
            seat = self.seat_layout[row][col]
            button = self.seats[(row, col)]
            if not seat.is_available():
                messagebox.showerror("Error", f"Seat {chr(65 + row)}{col + 1} is already booked!")
                return
            if button.cget("style") == "green.TButton":
                button.configure(style="yellow.TButton")
                seat.set_status("selected")
                self.selected_seats.append((row, col))
            elif button.cget("style") == "yellow.TButton":
                button.configure(style="green.TButton")
                seat.set_status("available")
                self.selected_seats.remove((row, col))
            logging.info(f"Seat {chr(65 + row)}{col + 1} toggled to {seat.status}")
        except Exception as e:
            logging.error(f"Seat selection failed: {e}")
            messagebox.showerror("Error", f"Failed to select seat: {e}")

    def confirm_booking(self) -> None:
        """Confirm and save selected seat bookings."""
        try:
            if not self.movie_var.get() or not self.showtime_var.get():
                messagebox.showerror("Error", "Please select a movie and showtime!")
                return
            if not self.selected_seats:
                messagebox.showerror("Error", "Please select at least one seat!")
                return
            user_name = self.user_name.get().strip()
            if not user_name:
                messagebox.showerror("Error", "Please enter your name!")
                return
            for row, col in self.selected_seats:
                seat = self.seat_layout[row][col]
                if seat.is_available():
                    raise ValueError(f"Seat {chr(65 + row)}{col + 1} is not selected")
                seat.set_status("booked")
                self.seats[(row, col)].configure(style="red.TButton")
                booking = Booking(self.movie_var.get(), self.showtime_var.get(), seat, user_name)
                self.db.save_booking(booking)
            messagebox.showinfo("Success", f"Booking confirmed for {user_name}!")
            self.selected_seats.clear()
            logging.info(f"Booking confirmed for {user_name}, {len(self.selected_seats)} seats")
        except Exception as e:
            logging.error(f"Booking confirmation failed: {e}")
            messagebox.showerror("Error", f"Booking failed: {e}")

    def clear_seat_layout(self) -> None:
        """Clear the current seat layout."""
        try:
            for widget in self.seat_frame.winfo_children():
                widget.destroy()
            self.seats.clear()
            self.selected_seats.clear()
            self.seat_layout = []
            logging.info("Seat layout cleared")
        except Exception as e:
            logging.error(f"Clear seat layout failed: {e}")

    def close_app(self) -> None:
        """Clean up and close the application."""
        try:
            self.db.close_connection()
            self.root.destroy()
            logging.info("Application closed")
        except Exception as e:
            logging.error(f"Application closure failed: {e}")

    def run(self) -> None:
        """Start the Tkinter event loop."""
        try:
            self.root.mainloop()
        except Exception as e:
            logging.error(f"Main loop failed: {e}")
            messagebox.showerror("Error", f"Application failed: {e}")

if __name__ == "__main__":
    try:
        app = CinemaApp()
        app.run()
    except Exception as e:
        logging.error(f"Application startup failed: {e}")
        print(f"Failed to start application: {e}")
